## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [3]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



'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'

In [4]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x109cd3370>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x109cd3f70>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Dishant and I am a Chief AI Engineer")])

AIMessage(content="Hello Dishant, it's a pleasure to meet you!\n\nBeing a Chief AI Engineer is a fascinating role. What kind of projects are you working on these days?  \n\nI'm eager to learn more about your work in the field of AI.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 23, 'total_tokens': 79, 'completion_time': 0.101818182, 'prompt_time': 0.000138419, 'queue_time': 0.018528100999999998, 'total_time': 0.101956601}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-0202afbe-a3f7-40b8-8b0a-2f9cf612e461-0', usage_metadata={'input_tokens': 23, 'output_tokens': 56, 'total_tokens': 79})

In [6]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Dishant and I am a Chief AI Engineer"),
        AIMessage(content="Hello Dishant! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Dishant, and you are a Chief AI Engineer.  \n\nIs there anything else you'd like to tell me about yourself or your work?  I'm happy to chat! 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 99, 'total_tokens': 144, 'completion_time': 0.081818182, 'prompt_time': 0.003974289, 'queue_time': 0.020004259, 'total_time': 0.085792471}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-6f5a9707-52b5-43f1-9be6-8fad7b30a842-0', usage_metadata={'input_tokens': 99, 'output_tokens': 45, 'total_tokens': 144})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [7]:
!pip install langchain_community

In [8]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [9]:
config={"configurable":{"session_id":"chat1"}}

In [10]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Dishant and I am a Chief AI Engineer")],
    config=config
)

In [11]:
response.content

"Hello Dishant, it's nice to meet you!  \n\nThat's a very impressive title.  What kind of AI projects are you working on these days? 🤖  I'm always interested to hear about what people are doing in the field.\n"

In [12]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Dishant.  \n\nI remembered it from our introduction! 😊  Is there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 94, 'total_tokens': 124, 'completion_time': 0.054545455, 'prompt_time': 0.002603643, 'queue_time': 0.019660125, 'total_time': 0.057149098}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-305a81fe-5c8a-4df0-8678-a88506452c70-0', usage_metadata={'input_tokens': 94, 'output_tokens': 30, 'total_tokens': 124})

In [13]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to know!\n"

In [14]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John! It's nice to meet you.  \n\nIs there anything I can help you with today?\n"

In [15]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John, you told me earlier! 😄  \n\nDo you have any other questions for me?\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [16]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [17]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Dishant")]})

AIMessage(content="Hi Dishant, it's nice to meet you! \n\nI'm ready to answer your questions to the best of my ability.  What can I help you with? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 31, 'total_tokens': 74, 'completion_time': 0.078181818, 'prompt_time': 0.000305579, 'queue_time': 0.020303361, 'total_time': 0.078487397}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-3dc3d30c-e359-43ad-89a6-7fa4f6b5ad88-0', usage_metadata={'input_tokens': 31, 'output_tokens': 43, 'total_tokens': 74})

In [18]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [19]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Dishant")],
    config=config
)

response

AIMessage(content="Hi Dishant, it's nice to meet you! \n\nI'm happy to help with any questions you have.  What can I do for you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 31, 'total_tokens': 72, 'completion_time': 0.074545455, 'prompt_time': 0.000315349, 'queue_time': 0.020475779, 'total_time': 0.074860804}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-95dbbafa-9984-452f-8247-7f32cfe53c06-0', usage_metadata={'input_tokens': 31, 'output_tokens': 41, 'total_tokens': 72})

In [20]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Dishant!  I remember that you told me.  😄 \n\n\nIs there anything else I can help you with? \n'

In [21]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [22]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Dishant")],"language":"Hindi"})
response.content

'नमस्ते Dishant! \n\nमुझे बहुत खुशी है कि आप मुझसे बात कर रहे हैं। मैं आपकी मदद करने के लिए यहाँ हूँ।  \n\nआप क्या जानना चाहते हैं या मेरी क्या मदद चाहिए? \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [23]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [24]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Dishant")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते Dishant! 👋 \n\nमुझे बहुत खुशी है कि आप मुझसे बात कर रहे हैं। \n\nआप मुझसे क्या पूछना चाहते हैं? 😊 \n\n'

In [25]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [26]:
response.content

'आपका नाम Dishant है। 😊  \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [27]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

/Users/prashantrao/Downloads/genAiNaik/chatbotwithMessageHistory/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [28]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal information, including your ice cream preferences. \n\nWhat's your favorite flavor? 😊🍦\n"

In [29]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "whats 2 + 2".  \n\n\n\nLet me know if you want to try another one! 😊 \n'

In [30]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [31]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I don't have access to any personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to know! 😊\n"

In [32]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. This means I don't know what math problem you might have asked before. \n\nIf you have a math problem you'd like help with, please ask! I'm ready. 😊  \n\n"